In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def resize_and_cut_image(image, new_width=800):
    new_image = image
    height, width = image.shape[:2]
    if width > new_width:
        aspect_ratio = height / float(width)
        new_height = int(new_width * aspect_ratio)
        new_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    new_image = image[int(height/3):, 0:width]
    return new_image

In [3]:
def filter_image(image):
    filtered_image = cv2.GaussianBlur(image, (3, 3), 0)
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened_image = cv2.filter2D(filtered_image, -1, kernel)

    return sharpened_image

In [4]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not load the image. Check image path")

    new_image = resize_and_cut_image(image)
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    filtered_image = filter_image(gray_image)
    
    return new_image, filtered_image

In [5]:
def get_contours(image):
    edges = cv2.Canny(image, 50, 150, L2gradient=True)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours

In [6]:
def get_plate(image, contours):
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    potentital_plate = None
    if len(sorted_contours) > 0:
        contour = sorted_contours[0]
        x, y, w, h = cv2.boundingRect(contour)
        
        aspect_ratio = w / float(h)
        area = cv2.contourArea(contour)
        
        if 3.0 <= aspect_ratio <= 6.0 and 1000 < area < 300000:
            potential_plate = contour

    if potential_plate is not None:
        x, y, w, h = cv2.boundingRect(potential_plate)
        license_plate = image[y:y+h, x:x+w]

        return license_plate

    raise ValueError("Could not find a license plate")

In [ ]:
for i in range(11):
    image, gray_image = load_and_preprocess_image(f"car_image_{10}.jpg")
    contours = get_contours(gray_image)
    plate = get_plate(image, contours)

    cv2.imshow("", plate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()